In [ ]:
# 데이터 처리 및 기본 라이브러리
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
tqdm.pandas() # pandas apply에 진행 상황 표시를 위함

# 머신러닝 모델 및 검증 도구
import lightgbm as lgbm
import xgboost as xgb
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score

# TF-IDF 및 차원 축소
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline

# 딥러닝(임베딩, PPL) 모델
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

In [ ]:
train = pd.read_csv('train.csv', encoding='utf-8-sig')
test = pd.read_csv('test.csv', encoding='utf-8-sig')
sample_submission = pd.read_csv('sample_submission.csv', encoding='utf-8-sig')

In [ ]:
# =======================================================================================
# === 이 셀은 시간이 매우 오래 걸립니다. 처음 한 번만 실행하여 특징을 저장하세요. ===
# =======================================================================================

# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- 모델 및 토크나이저 로드 ---
bert_tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
bert_model = AutoModel.from_pretrained('klue/bert-base').to(device)
ppl_tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2")
ppl_model = AutoModelForCausalLM.from_pretrained("skt/kogpt2-base-v2").to(device)

# --- 특징 생성 함수 정의 ---
def get_bert_embeddings(texts, batch_size=128):
    all_embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="BERT Embedding"):
        batch = texts[i:i+batch_size]
        batch_dict = bert_tokenizer(batch, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
        with torch.no_grad():
            outputs = bert_model(**batch_dict)
        embeddings = outputs.pooler_output
        all_embeddings.append(embeddings.cpu().numpy())
    return np.vstack(all_embeddings)

def get_perplexity(text):
    encodings = ppl_tokenizer(text, return_tensors="pt")
    max_length = ppl_model.config.max_position_embeddings
    stride = 512
    nlls = []
    for i in range(0, encodings.input_ids.size(1), stride):
        begin_loc, end_loc = max(i + stride - max_length, 0), min(i + stride, encodings.input_ids.size(1))
        trg_len = end_loc - i
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100
        with torch.no_grad():
            outputs = ppl_model(input_ids, labels=target_ids)
            neg_log_likelihood = outputs.loss * trg_len
        nlls.append(neg_log_likelihood)
    return torch.exp(torch.stack(nlls).sum() / end_loc).item()

# --- 훈련 데이터 특징 생성 ---
print("\n[훈련 데이터] 특징 생성 시작...")
X_train_title_emb = get_bert_embeddings(train['title'].tolist())
X_train_text_emb = get_bert_embeddings(train['full_text'].tolist())
X_train_embedding = np.concatenate([X_train_title_emb, X_train_text_emb], axis=1)

tqdm.pandas(desc="Train PPL")
train['ppl_full_text'] = train['full_text'].progress_apply(get_perplexity)
X_train_ppl = train[['ppl_full_text']].values

svd = TruncatedSVD(n_components=128, random_state=42)
tfidf = TfidfVectorizer(ngram_range=(1, 2), max_features=10000)
tfidf_pipeline = Pipeline([('tfidf', tfidf), ('svd', svd)])
X_train_tfidf = tfidf_pipeline.fit_transform(train['full_text'])

##정규화 진행하기 !!

X = np.concatenate([X_train_embedding, X_train_ppl, X_train_tfidf], axis=1)
y = train['generated'].values

# --- 테스트 데이터 특징 생성 (문단 단위) ---
print("\n[테스트 데이터] 특징 생성 시작 (문단 단위)...")
test = test.rename(columns={'paragraph_text': 'full_text'})

X_test_title_emb = get_bert_embeddings(test['title'].tolist())
X_test_text_emb = get_bert_embeddings(test['full_text'].tolist())
X_test_embedding = np.concatenate([X_test_title_emb, X_test_text_emb], axis=1)

tqdm.pandas(desc="Test PPL")
test['ppl_full_text'] = test['full_text'].progress_apply(get_perplexity)
X_test_ppl = test[['ppl_full_text']].values

X_test_tfidf = tfidf_pipeline.transform(test['full_text'])

X_test = np.concatenate([X_test_embedding, X_test_ppl, X_test_tfidf], axis=1)

# --- 파일 저장 ---
print("\n생성된 최종 특징 벡터들을 파일로 저장합니다...")
np.save('X_final_features.npy', X)
np.save('y_final_labels.npy', y)
np.save('X_test_final_paragraph.npy', X_test) # 문단 단위 테스트 특징 저장
print("저장 완료!")

Using device: cuda

[훈련 데이터] 특징 생성 시작...


Train PPL: 100%|██████████| 97172/97172 [1:51:07<00:00, 14.57it/s]



[테스트 데이터] 특징 생성 시작 (문단 단위)...


Test PPL: 100%|██████████| 1962/1962 [00:22<00:00, 87.12it/s]



생성된 최종 특징 벡터들을 파일로 저장합니다...
저장 완료!


In [ ]:
# =======================================================================================
# === 위 [Cell 3]에서 임베딩을 저장한 후, 다음부터는 이 셀만 실행하여 특징을 불러오세요. ===
# =======================================================================================

import numpy as np

# 저장된 파일 불러오기
X = np.load('X_final_features.npy')
y = np.load('y_final_labels.npy')
X_test = np.load('X_test_final_paragraph.npy')

print("저장된 특징 벡터를 성공적으로 불러왔습니다.")
print("훈련 데이터 특징 벡터 모양:", X.shape)
print("테스트 데이터 특징 벡터 모양:", X_test.shape)

저장된 특징 벡터를 성공적으로 불러왔습니다.
훈련 데이터 특징 벡터 모양: (97172, 1665)
테스트 데이터 특징 벡터 모양: (1962, 1665)


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgbm

# 이전에 Optuna로 직접 찾으신 최적의 하이퍼파라미터
best_params = {
    'learning_rate': 0.04312040147756817,
    'num_leaves': 41,
    'max_depth': 6,
    'subsample': 0.6968567940207964,
    'colsample_bytree': 0.8428253384396042,
    'lambda_l1': 9.288489210313957e-06,
    'lambda_l2': 0.0021254726288526147
}

# 고정 파라미터 추가
best_params['objective'] = 'binary'
best_params['metric'] = 'auc'
best_params['verbosity'] = -1
best_params['boosting_type'] = 'gbdt'
best_params['random_state'] = 42
best_params['n_estimators'] = 1000
best_params['device'] = 'gpu'

# --- K-Fold 교차 검증 시작 ---
N_SPLITS = 5
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

models = []
oof_preds = np.zeros(len(X))

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"===== Fold {fold+1} 시작 =====")
    X_train, y_train = X[train_idx], y[train_idx]
    X_val, y_val = X[val_idx], y[val_idx]

    # 위에서 정의한 best_params로 모델 학습
    model = lgbm.LGBMClassifier(**best_params)
    model.fit(X_train, y_train,
            eval_set=[(X_val, y_val)],
            eval_metric='auc',
            callbacks=[lgbm.early_stopping(100, verbose=False)])

    val_preds = model.predict_proba(X_val)[:, 1]
    oof_preds[val_idx] = val_preds
    models.append(model)

oof_auc = roc_auc_score(y, oof_preds)
print(f"\n최적 파라미터를 적용한 최종 OOF AUC: {oof_auc:.5f}")

===== Fold 1 시작 =====


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


===== Fold 2 시작 =====


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


===== Fold 3 시작 =====


...

===== Fold 4 시작 =====


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


===== Fold 5 시작 =====


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



최적 파라미터를 적용한 최종 OOF AUC: 0.91576


In [ ]:
# K-Fold로 학습된 모든 모델의 예측을 평균(앙상블)
test_preds_list = [model.predict_proba(X_test)[:, 1] for model in models]
test_preds_ensembled = np.mean(test_preds_list, axis=0)

# 예측 결과가 이미 문단 순서와 동일하므로 바로 할당
sample_submission['generated'] = test_preds_ensembled

# 제출 파일 생성
sample_submission.to_csv('submission_bert_lightgbm.csv', index=False)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
